# Table of Contents
* [Methodology](#Methodology)
	* [Community detection](#Community-detection)
	* [Selected features](#Selected-features)
		* [Common Neighbors](#Common-Neighbors)
		* [Adamic / Adar](#Adamic-/-Adar)
		* [Term Frequency / Inverse Document Frequency (TF-IDF)](#Term-Frequency-/-Inverse-Document-Frequency-%28TF-IDF%29)
		* [Common Referrers](#Common-Referrers)
		* [Publication Time Difference](#Publication-Time-Difference)
	* [Feature correlations](#Feature-correlations)
	* [Choice of classifier](#Choice-of-classifier)
		* [Feature importances](#Feature-importances)
		* [Estimating predictability](#Estimating-predictability)
* [Results for different communities](#Results-for-different-communities)
	* [Classifier trained on the 10 largest communities](#Classifier-trained-on-the-10-largest-communities)
		* [Feature importances](#Feature-importances)
		* [Predictability](#Predictability)
	* [Results for classifiers trained on individual communities](#Results-for-classifiers-trained-on-individual-communities)
		* [Differences in feature importances](#Differences-in-feature-importances)
		* [Differences in predictability](#Differences-in-predictability)
	* [Classifiers trained at different times in the network](#Classifiers-trained-at-different-times-in-the-network)
		* [Feature importances](#Feature-importances)
		* [Predictability](#Predictability)


# Methodology

## Community detection

To divide the verdicts into communities the Infomap method put forward by Rosvall et al (#cite-Rosvall) is used since it has already shown to group verdicts in ways that rougly corresponds to areas of law (#cite-Derlen).

Infomap is based on performing consucutive random walks on the network and using a compression of the resulting paths to create communities of nodes that are similar in the sense that they tend to be contained in the same random walks.
Using the Infomap implementation from the `igraph` Python package a total of 584 communities are found, however most communities are very small with only 20 communities having more than 50 members. 
![Size distribution of communities](Pictures/community sizes.png "Community size distribution")
Rosvall previously worked around this issue by resampling the network and merging communities based on additional iterations of the Infomap algorithm, however since we are interested in the differences between nodes we prefer having our communities as similar as possible, so we choose to work only on the nodes in the 10 largest communities.

| Community | 1   | 2   | 3   | 4   | 5   | 6   | 7   | 8   | 9   | 10 |
|-----------|-----|-----|-----|-----|-----|-----|-----|-----|-----|----|
| Size      | 169 | 161 | 127 | 125 | 118 | 116 | 111 | 108 | 101 | 85 |

## Selected features

The features chosen for predicting whether a potential link between two nodes exists or not are based both on structural information about the network such as the number of neighbors shared between two nodes or the similarity between the text content of the verdicts. Each feature will be introduced in short in the following section.

### Common Neighbors

Common neighbors or the intersection of the 1-neighborhoods of the nodes in
the candidate pair $(x, y)$ is the most commonly used index in link prediction
and it has been shown that the likelihood of a link between two nodes correlates
highly with the distance between the nodes as well as the number of neighbors
shared (Lü & Zhou, 2011). Formally the score is found by

$$
CN(x, y) = |\Gamma(x) \cup \Gamma(y)|
$$

where $\Gamma(x)$ is the function returning the 1-neighborhood of x.

### Adamic / Adar

The Adamic/Adar index was developed by Adamic and Adar (Adamic & Adar,
2003) to mine relationships on the web and has since been repurposed by many
authors as a general purpose link prediction index. Its rationale is that neighbors
that are shared by only a few nodes are more significant than neighbors that
are shared by many nodes. In link prediction for undirected networks this is

$$
AA(x, y) = \sum_{z \in \Gamma(x) \cup \Gamma(y)} \frac{1}{\log |\Gamma(z)|}
$$

where the $\Gamma(z)$ term is the number of node pairs for which z is a common
neighbor. Following the same thought of weighting by the inverse frequency the
following directed measures are used

### Term Frequency / Inverse Document Frequency (TF-IDF)

TF-IDF is used to estimate the similarity of two verdicts based on whether they use the same terminology. The details of the method won't be stated here, (an excellent reference can be found in [](#cite-Salton88)), but in short a vector of weighted word frequencies is generated for each verdict and for each possible pair of verdicts the cosine similarity is calculated. This results in a measure of how similar two verdicts is based on how often they use similar, rare words.

### Common Referrers

Common referrers is an expansion of
the common neighbors metric which
assumes that if x is connected to z
through u, then x is also more likely
to connect to other nodes that z is
connected to. Specific to the verdict citation networks this indicates
that verdicts that have the same citations are similar and that similar verdicts tend to have the same citations.
If a possible link between the node
pair (x, y) is being evaluated then the
common referrers index takes the following form
$$
CR(x, y) =\Gamma_{in}(y) \cap \Gamma_{in}(u) \ ∀ \ u \in \Gamma_{out}(x)
$$

![Common referrers](Pictures/Common referrers.png)
The neighborhood is also shown in the figure above where the common referrers are the
z-nodes marked in red for a potential connection between x and y. As can be
seen from the figure the considered nodes extends beyound the 1-neighborhood
of x, making common referrers the only quasi-global index considered in this
thesis.

### Publication Time Difference

One common factor among the citations made are that verdicts tend to make citations to recent verdicts instead of older verdicts. This is easily used for classification by adding the difference in publication time as a feature.

## Feature correlations

The correlations of the different features is shown in the picture below. Uncorrelated features are important to us when examining if different features are important for different communities, since correlation among features might show up as differences in feature importance between classifiers.

![Feature correlations](Pictures/feature correlations.png "Feature correlations")

The most correlated features are adamic/adar and common neighbors which is no surprise since they are built on the same mechanism.

## Choice of classifier

For a classifier we choose the random forest model, both because of its ease of implementation, its ability to capture non-linear relationsships and most significantly because the `sklearn` implementation is able to report feature importances which allows us to evaluate whether different feature are important for different communities.

### Feature importances

The feature importances are based on the depth of the splits the feature is involved in, where features that are used in splits higher in the tree affect more samples and is therefore seen as more important. This expected fraction of affected samples can be used to assign an importance value to each feature, with each value being between 0 and 1.0 and the sum of all importances being 1.0. (http://scikit-learn.org/stable/modules/ensemble.html#feature-importance-evaluation)

### Estimating predictability

To evaluate whether some communities are more predictable than others we use standard measures of classifier performance such as receiver operating characteristic (ROC) curves and precision-recall (PR) curves. Both measures are often used in machine learning since they are unaffected by the skew present in imbalanced datasets such as the set of verdicts that we are using.

Additionally we are using the median position of true links as well as the percentage of the top-20 potential links being true links. In both cases the position of the link is determined by the probability reported by the classifier.

# Results for different communities

The examination of the differences between communities is done in two ways: Creating a classifier that is trained on the potential links of the 10 largest communities and examining the differences in predictability between communities and training distinct classifiers on each community and comparing the classifiers. Training distinct classifiers has the advantage that it allows us to examine whether different features are important for different communities.

## Classifier trained on the 10 largest communities

### Feature importances

The feature importances for the random forest trained on the 10 largest communities can be seen in the picture below. While these cannot be used to find differences between communities it is a good baseline to use when comparing importances for classifiers trained on individual communities.
![Feature importances](Pictures/Feature importances full set.png)
The results show that the main features are the content similarity represented by TF-IDF, the structure found by Adamic/Adar and the time difference, with particularly common referrers and common neighbors less important. It is interesting that the most important features also show the least correlation, indicating that all features are relevant and hold information about link formation.

### Predictability

The predictability based on the classifier trained on the full set is reported both as overall predictability and split into predictability for each community. 
The reported statistics are the PR-AUC, ROC-AUC as well as the median position of true links and top-20 precision as mentioned in the methods section.

![Performance, full set](Pictures/performance full set.png)

As can be seen in the plot with the ROC and PR curves the classifiers performance is actually quite good when evaluated on the full network and able to achieve both precision and recall of about 0.4. In the plot below the median position of true links is shown and from this we can see that for approximately 54% of the nodes the median position of true links is in the top 20 positions.

![Median positions, full set](Pictures/position full set.png)

The summary statistics can be seen in table below. These statistics will act as the baseline to which we compare the results for the individual communities

 PR-AUC | ROC-AUC | Median position | In top 20 [%] 
--------|---------|-----------------|---------------
 0.327  | 0.888   | 18.5            | 54.6          
 
Below the difference between results for individual communities and the overall classifier is shown. For all statistics except the position larger means better. Apart from the differences compared to the results for the overall classifier absolute values for median position and the percentage of true links in the top 20 is reported as they have a more intuitive interpretation than the AUC values.

| Community | $\Delta$ PR-AUC | $\Delta$ ROC-AUC | $\Delta$ median position | $\Delta$ in top 20 [%] | Median position | In top 20 [%] |
|-----------|-----------------|------------------|--------------------------|------------------------|-----------------|---------------|
| 1         | 0.327           | 0.888            | 28.0                     | -18.64                 | 46.0            | 35.97         |
| 2         | -0.0566         | -0.0934          | 0.5                      | -1.46                  | 18.5            | 53.15         |
| 3         | 0.0537          | 0.0564           | 3.5                      | -5.46                  | 21.5            | 49.15         |
| 4         | -0.0168         | -0.0628          | -9.5                     | 12.85                  | 8.5             | 67.46         |
| 5         | -0.0284         | -0.0215          | -6.5                     | 10.59                  | 11.5            | 65.20         |
| 6         | 0.0122          | 0.0709           | -5.0                     | 8.73                   | 13.0            | 63.34         |
| 7         | 0.0437          | 0.0447           | -4.5                     | 5.09                   | 13.5            | 59.70         |
| 8         | 0.0007          | -0.0142          | 4.0                      | -5.86                  | 22.0            | 48.75         |
| 9         | -0.0256         | -0.0316          | -6.5                     | 11.34                  | 11.5            | 65.95         |
| 10        | 0.0214          | 0.0941           | -0.5                     | 1.96                   | 17.5            | 56.56         |

From the results it can be seen that there is quite a lot of difference in the predictability of different communities with for instance the first community performing significantly worse than the average while community 4 is far more predictable.

## Results for classifiers trained on individual communities

To get a clearer view of the difference between communities, random forest classifiers are trained and evaluated on each and their performance and feature importances examined.

### Differences in feature importances

Significant differences between the structure of communities could result in differences in the feature importances in the feature importances found by the random forest classifiers.

![Differences in feature importances](Pictures/feature importance differences.png)

From the plot of the percentage change in feature importances it can be seen that there are indeed differences between communities.

### Differences in predictability

Intuitively we would be able to get overall better performance out of classifiers trained on individual communities since we already showed that different feature impotances are important for different communities. As can be seen in the table below, this generally holds true with overall increases in both median position and the percentage of nodes having a median position of true links in the top 20.

| Community | $\Delta$ PR-AUC | $\Delta$ median position | $\Delta$ in top 20 | Median position | In top 20 [%] |
|-----------|-----------------|--------------------------|--------------------|-----------------|---------------|
| 1         | 0.0158137       | -1.500                   | 1.40               | 47.500          | 34.5          |
| 2         | -0.019093       | -6.000                   | 3.56               | 24.500          | 49.6          |
| 3         | 0.0053978       | -12.000                  | 7.61               | 33.500          | 41.5          |
| 4         | 0.0308701       | -3.500                   | 2.57               | 12.000          | 64.9          |
| 5         | -0.007484       | -1.750                   | 8.93               | 13.250          | 56.3          |
| 6         | -0.010325       | -1.250                   | 2.39               | 14.250          | 60.9          |
| 7         | 0.0016109       | -8.750                   | 7.29               | 22.250          | 52.4          |
| 8         | 0.0239969       | 6.125                    | -8.24              | 15.875          | 57.0          |
| 9         | 0.0041064       | 0.000                    | 1.97               | 11.500          | 63.9          |
| 10        | 0.03259         | 1.000                    | 1.24               | 16.500          | 55.3          |

# Results for classifiers trained at different times

To examine the evolution of the network over time classifiers are trained using snapshots of the network from different years and again examine predictability and feature importances.
Specifically we train classifiers on the potential links of nodes added in the intervals 2012-2008,2007-2003,2002-1998,1997-1993,1992-1988 and 1988-1994. As before predictability and feature importances are based on 5-fold cross validation.

## Feature importances

In the plot below the feature importances for each classifier is shown. Note that these are the actual importance values, not delta values as with the community.

![Feature importances](Pictures/feature importances year.png)

Some clear patterns are that features based on node properties such as TF-IDF and time difference are more important in the earlier years whereas structural features become more important in later years. This could indicate that the structure of the network becomes more regular over time.

## Predictability

An educated guess about the predictability of network would be that as the network develops structure over time it gets more predictable. Looking only at the predictability based on the ROC-AUC in the plot below this seems to be correct.
![Roc auc per year](Pictures/roc auc year.png)

If we additionally look at the median position of links the picture gets a bit murkier
| Year      | Median position | In top 20 | PR-AUC |
|-----------|-----------------|-----------|--------|
| 2012-2008 | 29.50           | 46.2      | 0.86   |
| 2007-2003 | 22.00           | 50.8      | 0.87   |
| 2002-1998 | 20.50           | 50.4      | 0.85   |
| 1997-1993 | 590.25          | 38.2      | 0.79   |
| 1992-1988 | 469.75          | 47.0      | 0.8    |
| 1987-1983 | 23.75           | 52.3      | 0.79   |
| 1982-1978 | 17.50           | 54.6      | 0.81   |

Notice that median position for 1992 and 1997 are far above anything else but that these numbers do not seem to follow the percentage of links in the top 20, nor the PR-AUC. As a gold standard of predictability it is probably best to choose ROC-AUC as both others depend on thresholds ( median position does not care about the shape of the distribution after and before the midpoint and in top 20 only cares about the top 20 positions).